In [3]:
# Paquetes

import numpy
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import (
        train_test_split,
        learning_curve,
        StratifiedKFold,
        cross_val_score
        )
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import unidecode
from functions_model import plots_freq_target,prepare_datasets,prepare_datasets_sinseg,get_file,plot_ROC,plot_confusion_matrix
from functions_model import plot_learning_curve,plot_ROC_kfolds

import json
import lightgbm as lgb
import pandas as pd
import numpy as np

from sklearn.metrics import roc_curve, mean_squared_error, roc_auc_score, precision_score 
from sklearn.metrics import precision_recall_curve, auc, confusion_matrix, recall_score
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report
#import scikitplot as skplt
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import xgboost as xgb

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'unidecode'

In [5]:
pip install unidecode

    100% |████████████████████████████████| 245kB 5.6MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier , export_graphviz
from sklearn.model_selection import train_test_split # Import train_test_split function
import os
import s3fs
import pyarrow.parquet as pq
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report
from sklearn.metrics import precision_recall_curve, auc, confusion_matrix, recall_score

pd.set_option('max_rows',100)
pd.set_option('max_columns',None)

# os.chdir('/../mnt')

def train_tree(mdt, y_var, params_dict):
    """
    Trains the tree to produce the profile
    :param params_dict:
    :return:
    """
    # Parameters
    predictive_vars = params_dict['predictive_cols']
    min_percent_leaf = params_dict['min_percent_leaf']

    mdt = mdt[predictive_vars]

    reg_tree = DecisionTreeClassifier(criterion="entropy", max_depth=3,min_samples_leaf=int(mdt.shape[0] * min_percent_leaf))

    reg_tree.fit(mdt, y_var)

    return reg_tree


def produce_dot_file(trained_tree, file_name):
    full_path = params_dict['output_path'] + file_name
    export_graphviz(trained_tree, feature_names=params_dict['predictive_cols'],
                    out_file=full_path,
                    filled=True, proportion=True,
                    rounded=True)


def main(mdt, y_var):
    name = 'tree_'+params_dict['segment_name']+'_V0.dot'
    my_tree = train_tree(mdt, y_var, params_dict)
    produce_dot_file(my_tree, name)
    print('Tree produced')
    return my_tree



In [ ]:
# Definir parámetros
params_dict = {'min_percent_leaf': 0.05,
               'input_path':'s3://adl-refined-dev-popular/parquet/TC_adquisicion/total_tdc_perf',
              'output_path':'/mnt/work/CU_adquisicionTC/Notebooks/creacion modelo/resultados_arbolV0/',
              'segment_name':'perfilados'}

In [ ]:
fs = s3fs.S3FileSystem()

# Leyendo base
input = params_dict['input_path']
dataset = pq.ParquetDataset(input, filesystem=fs)
table = dataset.read()
mdt = table.to_pandas()

del input, dataset, table

# Definiendo Var Obj'var_final'etivo
y_var = '30first_use'


In [ ]:
mdt.shape

In [ ]:
mdt.columns

In [ ]:
np.random.seed(seed=1234) # Semilla
## Muestra del 50%
#Se generan numeros aleatorios entre 0 y 1
#mdt['ran']=np.random.random(size=mdt.shape[0])

#Se seleccion train : 50% 
mdt_v0=mdt.copy()

# quitando otras variables VO
mdt_v0.drop(columns=['var_final','mes_campaña','venta', 'fecha_activo', 'fecha_buro', 'fecha_data', 'fecha_envio',
                     'tipo_campana', 'periodo','cedulaenc','tipo_cliente','tipo_id','derogatorio',
                    'fecha_real','fecha_mas_antigua_apertura','timestamp','lag_t_6_saldo_tot_tot_SMLV', 
                     'lag_t_1_cuota_tot_LB_SMLV', 'lag_t_6_cuota_tot_LB_SMLV', 'lag_t_6_cuota_tot_tot_SMLV', 
                     'lag_t_1_amortizacion_prom_LB', 'lag_t_6_amortizacion_prom_LB', 'lag_t_1_amortizacion_prom_tot', 
                     'lag_t_6_amortizacion_prom_tot', 'lag_t_6_saldo_tot_LB_SMLV', 'lag_t_1_saldo_tot_tot_SMLV', 
                     'lag_t_1_cuota_tot_tot_SMLV', 'lag_t_6_cupo_tot_LB_SMLV', 'lag_t_1_cupo_tot_tot_SMLV', 'lag_t_6_cupo_tot_tot_SMLV'], inplace = True)



In [ ]:
## Vars modelo:

top_vars = [
'acep_oferta_prev',
'num_act_utl_meses',
'num_camp_ult_meses',
'num_meses_ult_camp',
'num_no_aceptado',
'num_lib_solicitadas',
'prom_monto_solicitado',
'prom_monto_aprobado',
'prom_monto_desembolsado',
'prom_monto_novado',
'prom_prop_desembolso',
'prom_n_cuotas',
'prom_valor_cuota',
'prom_tasa',
'prom_dias_procesamiento',
'max_dias_procesamiento',
'n_novaciones',
'sector_ultlibranza',
'subsector_ultlibranza',
'tipo_pagaduria_ultlibranza',
'pasv_num_ctas_vig_tot',
'pasv_num_ctas_vig_cc',
'pasv_num_ctas_vig_ca',
'pasv_num_ctas_vig_cdt',
'pasv_dias_desde_ultima_trans',
'pasv_antig_total',
'pasv_num_meses_ult_apertura',
'pasv_saldo_tot_fin_mes_smlv',
'pasv_saldo_cc_fin_mes_smlv',
'pasv_saldo_ca_fin_mes_smlv',
'pasv_saldo_cdt_fin_mes_smlv',
'pasv_saldo_min_fin_mes_smlv',
'pasv_saldo_min_cc_fin_mes_smlv',
'pasv_saldo_min_ca_fin_mes_smlv',
'pasv_saldo_min_cdt_fin_mes_smlv',
'pasv_saldo_max_fin_mes_smlv',
'pasv_saldo_max_cc_fin_mes_smlv',
'pasv_saldo_max_ca_fin_mes_smlv',
'pasv_saldo_max_cdt_fin_mes_smlv',
'pasv_saldo_total_1mes_atras',
'pasv_saldo_cc_1mes_atras',
'pasv_saldo_ca_1mes_atras',
'pasv_saldo_cdt_1mes_atras',
'pasv_saldo_total_3mes_atras',
'pasv_saldo_cc_3mes_atras',
'pasv_saldo_ca_3mes_atras',
'pasv_saldo_cdt_3mes_atras',
'pasv_saldo_total_6mes_atras',
'pasv_saldo_cc_6mes_atras',
'pasv_saldo_ca_6mes_atras',
'pasv_saldo_cdt_6mes_atras',
'num_tx_ult_1mes_pasv',
'num_tx_ult_2meses_pasv',
'num_tx_ult_3meses_pasv',
'num_tx_ult_4meses_pasv',
'num_tx_ult_6meses_pasv',
'prom_tx_ult_1meses_pasv',
'prom_tx_ult_2meses_pasv',
'prom_tx_ult_3meses_pasv',
'prom_tx_ult_4meses_pasv',
'prom_tx_ult_6meses_pasv',
'mode_dia_pasv',
'estado_cta_actual_pasv',
'tipo_transaccion',
'sum_tx_linea_pasv',
'sum_tx_nolinea_pasv',
'fecha_ult_tx_pasv',
'dias_desde_ult_tx_pasv',
'num_tx_ult_1mes_cte',
'num_tx_ult_2meses_cte',
'num_tx_ult_3meses_cte',
'num_tx_ult_4meses_cte',
'num_tx_ult_6meses_cte',
'prom_tx_ult_1meses_cte',
'prom_tx_ult_2meses_cte',
'prom_tx_ult_3meses_cte',
'prom_tx_ult_4meses_cte',
'prom_tx_ult_6meses_cte',
'mode_dia_cte',
'estado_cta_actual_cte',
'tipo_transaccion_cte',
'sum_tx_linea_cte',
'fecha_ult_tx_cte',
'dias_desde_ult_tx_cte',
'num_tx_ult_1mes_cdt',
'num_tx_ult_2meses_cdt',
'num_tx_ult_3meses_cdt',
'num_tx_ult_4meses_cdt',
'num_tx_ult_6meses_cdt',
'prom_tx_ult_1meses_cdt',
'prom_tx_ult_2meses_cdt',
'prom_tx_ult_3meses_cdt',
'prom_tx_ult_4meses_cdt',
'prom_tx_ult_6meses_cdt',
'mode_dia_cdt',
'estado_cta_actual_cdt',
'tipo_transaccion_cdt',
'sum_tx_nolinea_cdt',
'fecha_ult_tx_cdt',
'dias_desde_ult_tx_cdt',
'num_tx_ult_1mes_ahr',
'num_tx_ult_2meses_ahr',
'num_tx_ult_3meses_ahr',
'num_tx_ult_4meses_ahr',
'num_tx_ult_6meses_ahr',
'prom_tx_ult_1meses_ahr',
'prom_tx_ult_2meses_ahr',
'prom_tx_ult_3meses_ahr',
'prom_tx_ult_4meses_ahr',
'prom_tx_ult_6meses_ahr',
'mode_dia_ahr',
'estado_cta_actual_ahr',
'tipo_transaccion_ahr',
'sum_tx_linea_ahr',
'sum_tx_nolinea_ahr',
'fecha_ult_tx_ahr',
'dias_desde_ult_tx_ahr'
]

In [ ]:
mdt_v0 = mdt_v0[['id_cliente','30first_use']+top_vars]

In [ ]:
cat_cols = list(mdt_v0.dtypes[mdt_v0.dtypes=='object'].index)
cat_cols

In [ ]:
mdt_v0['sector_ultlibranza'].value_counts()

In [ ]:
# feature engineering

# mdt_v0['ciudad_de_expedicion_cat'] = (mdt_v0['ciudad_de_expedicion'] == 'BOGOTA D.C.').astype('int')
# mdt_v0['genero_cat'] = (mdt_v0['genero'] == 'F').astype('int')
# mdt_v0['peor_califi_trim_1_endeud_cat'] = (mdt_v0['peor_califi_trim_1_endeud'] == 'mantiene').astype('int')
# mdt_v0['peor_calif_trim_2_endeud_cat'] = (mdt_v0['peor_calif_trim_2_endeud'] == 'mantiene').astype('int')
# mdt_v0['rango_aproximado_edad_cat'] = (mdt_v0['rango_aproximado_edad'] == 'mantiene').astype('int')
# mdt_v0['marca_derogatorio_cat'] = (mdt_v0['marca_derogatorio'] == 'aprobado').astype('int')
# mdt_v0['tdc_altura_maxima_de_mora_was_is_cat'] = (mdt_v0['tdc_altura_maxima_de_mora_was_is'] == 'mantiene').astype('int')
# mdt_v0['cartera_banca_alt_max_de_mora_was_is_cat'] = (mdt_v0['cartera_banca_alt_max_de_mora_was_is'] == 'mantiene').astype('int')
# mdt_v0['cartera_coope_alt_max_de_mora_was_is_cat'] = (mdt_v0['cartera_coope_alt_max_de_mora_was_is'] == 'mantiene').astype('int')
# mdt_v0['cartera_hipote_alt_max_de_mora_was_is_cat'] = (mdt_v0['cartera_hipote_alt_max_de_mora_was_is'] == 'mantiene').astype('int')
# mdt_v0['moramax_mercado_was_is_cat'] = (mdt_v0['moramax_mercado_was_is'] == 'mantiene').astype('int')
# mdt_v0['was_is_LB_cat'] = (mdt_v0['was_is_LB'] == 'mantiene').astype('int')
# mdt_v0['was_is_TC_cat'] = (mdt_v0['was_is_TC'] == 'mantiene').astype('int')
# mdt_v0['was_is_NV_cat'] = (mdt_v0['was_is_NV'] == 'mantiene').astype('int')
# mdt_v0['was_is_CH_cat'] = (mdt_v0['was_is_CH'] == 'mantiene').astype('int')
# mdt_v0['was_is_CE_cat'] = (mdt_v0['was_is_CE'] == 'mantiene').astype('int')
# mdt_v0['was_is_LE_cat'] = (mdt_v0['was_is_LE'] == 'mantiene').astype('int')
# mdt_v0['was_is_ME_cat'] = (mdt_v0['was_is_ME'] == 'mantiene').astype('int')
# mdt_v0['was_is_SO_cat'] = (mdt_v0['was_is_SO'] == 'mantiene').astype('int')
# mdt_v0['was_is_CC_cat'] = (mdt_v0['was_is_CC'] == 'mantiene').astype('int')
# mdt_v0['was_is_TC_X_cat'] = (mdt_v0['was_is_TC_X'] == 'mantiene').astype('int')
# mdt_v0['was_is_TC_T_cat'] = (mdt_v0['was_is_TC_T'] == 'mantiene').astype('int')
# mdt_v0['was_is_tot_cat'] = (mdt_v0['was_is_tot'] == 'mantiene').astype('int')


mdt_v0['tiene_sector_libranza'] = (mdt_v0['sector_ultlibranza'].notnull()).astype('int')
mdt_v0['tiene_subsector_libranza'] = (mdt_v0['subsector_ultlibranza'] == 'mantiene').astype('int')
mdt_v0['tiene_pagaduria_libranza'] = (mdt_v0['tipo_pagaduria_ultlibranza'] == 'mantiene').astype('int')
mdt_v0['estado_cta_actual_pasv1'] = (mdt_v0['estado_cta_actual_pasv'] == '1').astype('int')

mdt_v0['tipo_transaccion_pasv2'] = (mdt_v0['tipo_transaccion'] == '2').astype('int')
mdt_v0['estado_cta_actual_pasv1'] = (mdt_v0['estado_cta_actual_pasv'] == '1').astype('int')

mdt_v0['tipo_transaccion_pasv2'] = (mdt_v0['tipo_transaccion'] == '2').astype('int')
mdt_v0['estado_cta_actual_pasv1'] = (mdt_v0['estado_cta_actual_pasv'] == '1').astype('int')

mdt_v0['tipo_transaccion_pasv2'] = (mdt_v0['tipo_transaccion'] == '2').astype('int')
mdt_v0['estado_cta_actual_pasv1'] = (mdt_v0['estado_cta_actual_pasv'] == '1').astype('int')

mdt_v0['tipo_transaccion_pasv2'] = (mdt_v0['tipo_transaccion'] == '2').astype('int')
mdt_v0['estado_cta_actual_pasv1'] = (mdt_v0['estado_cta_actual_pasv'] == '1').astype('int')



In [ ]:
pd.get_dummies(mdt_v0[['mode_dia_pasv',
'mode_dia_cte',
'estado_cta_actual_cte',
'tipo_transaccion_cte',
'mode_dia_cdt',
'estado_cta_actual_cdt',
'tipo_transaccion_cdt',
'fecha_ult_tx_cdt',
'mode_dia_ahr',
'estado_cta_actual_ahr',
'tipo_transaccion_ahr',
'fecha_ult_tx_ahr']]).head(20)

In [ ]:
mdt_v0['fecha_ult_tx_pasv'].value_counts()

In [ ]:
mdt_v0.drop(columns=cat_cols, inplace = True)

In [ ]:
mdt_v0.set_index(['id_cliente'], inplace=True)

In [ ]:
np.random.seed(seed=1234) # Semilla

#Se generan numeros aleatorios entre 0 y 1
mdt_v0['ran2']=np.random.random(size=mdt_v0.shape[0])

#Se seleccion train : 50% 
mdt_train=mdt_v0[mdt_v0['ran2']<=0.7]
mdt_test=mdt_v0[mdt_v0['ran2']>0.7]

###
y = mdt_train['30first_use']
y_test = mdt_test['30first_use']

mdt_train.drop(columns=['30first_use','ran2'],inplace=True)
mdt_test.drop(columns=['30first_use','ran2'],inplace=True)

In [ ]:
cols_modelo = mdt_train.columns
params_dict['predictive_cols'] = cols_modelo

In [ ]:
np.random.seed(seed=1234) # Semilla
# Split dataset into training set and test set
X_train, X_val, y_train, y_val = train_test_split(mdt_train, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [ ]:
y_train.value_counts(normalize=True)*100

In [ ]:
y_val.value_counts(normalize=True)*100

In [ ]:
y_test.value_counts(normalize=True)*100

In [ ]:
#vars_final = list(cols_modelo)
vars_final = ['acep_oferta_prev', 'num_act_utl_meses', 'num_camp_ult_meses', 'num_meses_ult_camp', 'num_no_aceptado', 
'num_lib_solicitadas', 'prom_monto_solicitado', 'prom_monto_aprobado', 'prom_monto_desembolsado', 'prom_monto_novado', 
'prom_prop_desembolso', 'prom_n_cuotas', 'prom_valor_cuota', 'prom_tasa', 'prom_dias_procesamiento', 'max_dias_procesamiento', 
'n_novaciones', 'pasv_num_ctas_vig_tot', 'pasv_num_ctas_vig_cc', 'pasv_num_ctas_vig_ca', 'pasv_num_ctas_vig_cdt', 
'pasv_dias_desde_ultima_trans', 'pasv_antig_total', 'pasv_num_meses_ult_apertura', 'pasv_saldo_tot_fin_mes_smlv', 
'pasv_saldo_cc_fin_mes_smlv', 'pasv_saldo_ca_fin_mes_smlv', 'pasv_saldo_cdt_fin_mes_smlv', 'pasv_saldo_min_fin_mes_smlv', 
'pasv_saldo_min_cc_fin_mes_smlv', 'pasv_saldo_min_ca_fin_mes_smlv', 'pasv_saldo_min_cdt_fin_mes_smlv', 
'pasv_saldo_max_fin_mes_smlv', 'pasv_saldo_max_cc_fin_mes_smlv', 'pasv_saldo_max_ca_fin_mes_smlv', 
'pasv_saldo_max_cdt_fin_mes_smlv', 'pasv_saldo_total_1mes_atras', 'pasv_saldo_cc_1mes_atras', 'pasv_saldo_ca_1mes_atras', 
'pasv_saldo_cdt_1mes_atras', 'pasv_saldo_total_3mes_atras', 'pasv_saldo_cc_3mes_atras', 'pasv_saldo_ca_3mes_atras', 
'pasv_saldo_cdt_3mes_atras', 'pasv_saldo_total_6mes_atras', 'pasv_saldo_cc_6mes_atras', 'pasv_saldo_ca_6mes_atras', 
'pasv_saldo_cdt_6mes_atras', 'tiene_sector_libranza', 'tiene_subsector_libranza', 'tiene_pagaduria_libranza']

### Modelo V0 1200 variables

In [ ]:
import shap
import scikitplot as skplt
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

#Se ajusta el modelo con las mejores variables best_20_v3
#'multi:softmax'
xgb_model = xgb.XGBClassifier(objective = 'binary:logistic', 
                              seed = 1,  
                              max_depth = 5, 
                              colsample_bytree = 0.8,
                              learning_rate = 0.1,
                              gamma=0.2,
                              scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int),
                              n_estimators=150, min_child_weight = 5)

xgb_model.fit(X_train[vars_final], y_train, 
              eval_set = [(X_train[vars_final], y_train), (X_val[vars_final], y_val)],
              early_stopping_rounds=150)

In [ ]:
# make predictions for test data
y_pred = model.predict(X_val)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_train_pred = model.predict(X_train)

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

In [ ]:
#Matriz de confusión
predict_test = (probs_test[:, 1] > 0.00036)*1
metrics.confusion_matrix(y_test,predict_test)

In [ ]:
# split data into train and test sets
seed = 123
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(mdt_train, y, test_size=test_size, random_state=seed)
#fit model no training data
model = xgb.XGBClassifier(objective = 'binary:logistic', 
                              seed = 1,  
                              max_depth = 5, 
                              colsample_bytree = 0.8,
                              learning_rate = 0.1,
                              gamma=0.2,
                              scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int),
                              n_estimators=150, min_child_weight = 5)

model.fit(X_train, y_train)

print(model)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_train_pred = model.predict(X_train)

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

In [ ]:
#evaluate predictions
accuracy = accuracy_score(y_test, predictions)
accuracy_train = accuracy_score(y_train, y_train_pred)
precision=precision_score(y_test, predictions)
precision_train=precision_score(y_train, y_train_pred)
recall=recall_score(y_test, predictions)
recall_train=recall_score(y_train, y_train_pred)
roc=roc_auc_score(y_test,predictions)
kappa = cohen_kappa_score(y_test,predictions)
kappa_train = cohen_kappa_score(y_train,y_train_pred)
print("AUC-ROC: %.2f%% " % (roc *100))
print("Precision: %.2f%% " % (precision *100))
print("Sensibilidad (Recall): %.2f%% " % (recall * 100))
print("Kappa_train: %.2f%% " % (kappa_train * 100))
print("Kappa_test: %.2f%% " % (kappa * 100))
print("Accuracy_train: %.2f%% " % (accuracy_train * 100.0))
print("Accuracy_test: %.2f%% " % (accuracy * 100.0))
print("Especificidad: %.2f%% " % round(cf_matrix[0,0]/cf_matrix[0,:].sum()*100,2) )
#print("Precision_train: %.2f%% " % (precision_train *100))
#print("Recall_train: %.2f%% " % (recall_train * 100))

In [ ]:
#Accuracy
mdt_test['probs'] = probs_test[:, 1]
metrics.accuracy_score(y_test, predict_test)

In [ ]:
#Recall
metrics.recall_score(y_test, predict_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager
%matplotlib inline

# Graficar probabilidades según Fraude/No Fraude observado (real)

subset_si = mdt_test[y_test == 1]

subset_no = mdt_test[y_test == 0]

sns.distplot(subset_no[['probs']], hist = True, norm_hist = True, bins = 10, kde = False, kde_kws = {'linewidth' : 3},
            label = 'No usa', color = 'green')

sns.distplot(subset_si[['probs']], hist = True, norm_hist = True, bins = 10, kde = False, kde_kws = {'linewidth' : 3},
            label = 'Si usa', color = 'red')

plt.legend(prop = {'size' : 10})
plt.title('Distribución de scores según Primer uso')
plt.xlabel('Probabilidad')
plt.ylabel('Porcentaje')
plt.show()

In [ ]:
skplt.metrics.plot_cumulative_gain(y_test, probs_test)

In [ ]:
shap_values = shap.TreeExplainer(xgb_model).shap_values(mdt_test[vars_final])
shap.summary_plot(shap_values, mdt_test[vars_final])


In [ ]:
vars_final = ['acep_oferta_prev', 'num_act_utl_meses', 'num_camp_ult_meses', 'num_meses_ult_camp', 'num_no_aceptado', 
'num_lib_solicitadas', 'prom_monto_solicitado', 'prom_monto_aprobado', 'prom_monto_desembolsado', 'prom_monto_novado', 
'prom_prop_desembolso', 'prom_n_cuotas', 'prom_valor_cuota', 'prom_tasa', 'prom_dias_procesamiento', 'max_dias_procesamiento', 
'n_novaciones', 'pasv_num_ctas_vig_tot', 'pasv_num_ctas_vig_cc', 'pasv_num_ctas_vig_ca', 'pasv_num_ctas_vig_cdt', 
'pasv_dias_desde_ultima_trans', 'pasv_antig_total', 'pasv_num_meses_ult_apertura', 'pasv_saldo_tot_fin_mes_smlv', 
'pasv_saldo_cc_fin_mes_smlv', 'pasv_saldo_ca_fin_mes_smlv', 'pasv_saldo_cdt_fin_mes_smlv', 'pasv_saldo_min_fin_mes_smlv', 
'pasv_saldo_min_cc_fin_mes_smlv', 'pasv_saldo_min_ca_fin_mes_smlv', 'pasv_saldo_min_cdt_fin_mes_smlv', 
'pasv_saldo_max_fin_mes_smlv', 'pasv_saldo_max_cc_fin_mes_smlv', 'pasv_saldo_max_ca_fin_mes_smlv', 
'pasv_saldo_max_cdt_fin_mes_smlv', 'pasv_saldo_total_1mes_atras', 'pasv_saldo_cc_1mes_atras', 'pasv_saldo_ca_1mes_atras', 
'pasv_saldo_cdt_1mes_atras', 'pasv_saldo_total_3mes_atras', 'pasv_saldo_cc_3mes_atras', 'pasv_saldo_ca_3mes_atras', 
'pasv_saldo_cdt_3mes_atras', 'pasv_saldo_total_6mes_atras', 'pasv_saldo_cc_6mes_atras', 
'pasv_saldo_cdt_6mes_atras', 'tiene_sector_libranza', 'tiene_subsector_libranza', 'tiene_pagaduria_libranza']

In [ ]:
mdt['num_lib_solicitadas']

In [ ]:
shap.summary_plot(shap_values, mdt_test[vars_final], plot_type = 'bar')

## Grid Search

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=150, max_depth=5,
 scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int),
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)

In [ ]:
gsearch1.best_score_,gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int)
                                                      ,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int)
                                                      ,seed=27), 
param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=5, gamma=0.2, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int)
                                                      ,seed=27), 
param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int)
                                                      ,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
#kappa_scorer=make_scorer(cohen_kappa_score)
scoring = {'Precision': 'precision', 'Recall': 'recall', 'Roc':'roc_auc'}
param_test1 = {
    'max_depth': range(4,12,2),
    'min_child_weight':range(4,8,2),
    'learning_rate': [0.01, 0.1, 0.2]
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.1, n_estimators=150, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4,                                       
                                                      scale_pos_weight=np.ceil(y_train.shape[0]/y_train.sum()).astype(int), seed=None), 
                       param_grid = param_test1, scoring=scoring, n_jobs=4,iid=False, cv=3)
gsearch1.fit(X_train,y_train)

### Modelo con variables más importantes (top 60)

In [ ]:
def train_eval_xgboost(df, features, y, model_num, df_importance):
    
    """
    Función que realiza lo siguiente: 
    1. A partir de Train, realiza un sampling aleatorio 70-30 para Train y Validación
    2. Entrena un modelo XGBoost con la data de Train, usando tanto la data Train como Validación 
        para evaluar (métrica: AUC)
    3. Calcula mean(|SHAP Value|) por variable para base de Train y guarda el valor en df_importance
    4. Calcula auc, accuracy y recall sobre base de Test y guarda los valores en df_metricas, utilizando
    un threshold determinado por el usuario (0.5 por default)
    """
    
    X_train, X_val, y_train, y_val = train_test_split(df[features], y, test_size = .3)
    
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    
    print(model_num)
    
    "Entrenar modelo XGBoost"
    
    xgb_model = xgb.XGBClassifier(objective = 'binary:logistic', 
                             max_depth = 3, 
                             colsample_bytree = 0.7,
                             learning_rate = 0.1, 
                             n_estimators = 1000)

    xgb_model.fit(X_train, y_train, 
                  eval_set = [(X_train, y_train), (X_val, y_val)], 
                  early_stopping_rounds = 500, 
                  verbose = False)
    
    print('Modelo entrenado')
    
    "Calcular y guardar SHAP Values"
    
    shap_values = shap.TreeExplainer(xgb_model).shap_values(df[features])
    shap_feature = np.abs(shap_values).mean(axis = 0)
    
    df_importance['mean_SHAP_'+model_num] = shap_feature
    
    print('SHAP Values guardados')

In [ ]:
features_x = cols_modelo
feature_importances = pd.DataFrame({'feature_name' : features_x}, columns = ['feature_name'])

In [ ]:
#Se ejecuta 10 veces la función train_eval_xgboost para la base train (80% de la base total de desarrollo) 
# y se calculan los shap values (para las 10 iteraciones) que quedan guardados en el feature_importances

#for i in range(1, 3):
for i in range(1, 11):
    
    model_num = 'model_'+str(i)
    
    train_eval_xgboost(mdt_train, features_x, y, model_num, feature_importances)

In [ ]:
#A partir de los shap values guardados en feature_importances, se otorga un ranking a las variables y se ordenan según ese ranking

for col in feature_importances.columns[1:]:
    
    feature_importances[col + '_rank'] = feature_importances[col].rank(ascending = False)

In [ ]:
# Se seleccionan las 20 mejores variables según ranking anterior
# OJO!!!!!En feature_importances.columns[5:9] se debe poner la posicion de la primera variable _rank hasta la posicion de la ultima variable _rank
principales = []
for col in feature_importances.columns[11:]:
    
    best_60 = [x for x in feature_importances[feature_importances[col] < 100].feature_name.values]
    principales.extend(best_60)

In [ ]:
# Se seleccionan las 20 mejores variables según ranking anterior
# OJO!!!!!En feature_importances.columns[5:9] se debe poner la posicion de la primera variable _rank hasta la posicion de la ultima variable _rank
peores = []
for col in feature_importances.columns[11:]:
    
    peorest = [x for x in feature_importances[feature_importances[col] > 500].feature_name.values]
    peores.extend(peorest)

In [ ]:
peores

In [ ]:
best_60

In [ ]:
#Se ajusta el modelo con las mejores variables best_20_v3
#'multi:softmax'
xgb_model_v2 = xgb.XGBClassifier(objective = 'binary:logistic', 
                              seed = 1,  
                              max_depth = 3, 
                              colsample_bytree = 0.8,
                              learning_rate = 0.1, 
                              n_estimators = 1000, min_child_weight = 3)

xgb_model_v2.fit(X_train[best_60], y_train, 
              eval_set = [(X_train[best_60], y_train), (X_val[best_60], y_val)],
              early_stopping_rounds=300)

In [ ]:
# Predecir probabilidades para data test (20%)

probs_test = xgb_model_v2.predict_proba(mdt_test[best_60])

fpr, tpr, thresholds = metrics.roc_curve(y_test, probs_test[:, 1])

#AUC
metrics.auc(fpr, tpr)

In [ ]:
#Matriz de confusión
predict_test = (probs_test[:, 1] > 0.045)*1
metrics.confusion_matrix(y_test,predict_test)


In [ ]:
#Accuracy
mdt_test['probs'] = probs_test[:, 1]
metrics.accuracy_score(y_test, predict_test)


In [ ]:
#Recall
metrics.recall_score(y_test, predict_test)

In [ ]:
# Graficar probabilidades según Fraude/No Fraude observado (real)

subset_si = mdt_test[y_test == 1]

subset_no = mdt_test[y_test == 0]

sns.distplot(subset_no[['probs']], hist = True, norm_hist = True, bins = 10, kde = False, kde_kws = {'linewidth' : 3},
            label = 'No usa < 30D', color = 'green')

sns.distplot(subset_si[['probs']], hist = True, norm_hist = True, bins = 10, kde = False, kde_kws = {'linewidth' : 3},
            label = 'Si usa < 30D', color = 'red')

plt.legend(prop = {'size' : 10})
plt.title('Distribución de scores según Primer uso')
plt.xlabel('Probabilidad')
plt.ylabel('Porcentaje')
plt.show()

In [ ]:
skplt.metrics.plot_cumulative_gain(y_test, probs_test)

In [ ]:
shap_values = shap.TreeExplainer(xgb_model_v2).shap_values(mdt_test[best_60])
shap.summary_plot(shap_values, mdt_test[best_60])

In [ ]:
shap.summary_plot(shap_values, mdt_test[best_60], plot_type = 'bar')

In [ ]:
## Correr arbol
#main(X_train,y_train)


In [ ]:
# Testing tree
my_tree(X_test)

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))